<a href="https://colab.research.google.com/github/ShahzaibSE/langgraph-e2e-solutions/blob/main/edges_and_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is LangGraph?

LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence.

LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions.

As a very low-level framework, it provides fine-grained control over both the flow and state of your application, crucial for creating reliable agents.

Additionally, LangGraph includes built-in persistence, enabling advanced human-in-the-loop and memory features.

Key Features¶
- **Cycles and Branching:** Implement loops and conditionals in your apps.
- **Persistence:** Automatically save state after each step in the graph. Pause and resume the graph execution at any point to support error recovery, human-in-the-loop workflows, time travel and more.
- **Human-in-the-Loop:** Interrupt graph execution to approve or edit next action planned by the agent.
- **Streaming Support:** Stream outputs as they are produced by each node (including token streaming).
- **Integration with LangChain:** LangGraph integrates seamlessly with LangChain and LangSmith (but does not require them).

https://langchain-ai.github.io/langgraph/

https://chatgpt.com/share/66f9d1bf-58e0-8002-84d0-1c1f8d5b4389

In [1]:
%%capture --no-stderr
!pip install --quiet -U langgraph

### **State**



In [4]:
from langgraph import nodegraph

ImportError: cannot import name 'nodegraph' from 'langgraph' (unknown location)